In [21]:
import cv2
import numpy as np
import os 
import json

In [22]:
JSON_PREDICTIONS_PATH = '/home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/inference-result/instance_segmentation.json'
ORIGINAL_IMAGES_DIR = '/home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/patches'
OUTPUT_OVERLAY_DIR = '/home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay'
os.makedirs(OUTPUT_OVERLAY_DIR, exist_ok=True)

In [23]:
NUCLEI_TYPE_COLORS = {
    "Connective": (255, 0, 0),    # Blue
    "Neoplastic": (0, 0, 255),    # Red
    "Inflammatory": (0, 255, 0),  # Green
    "Epithelial": (255, 255, 0),  # Cyan
    "Dead": (0, 255, 255),        # Yellow
}

In [24]:
print(f"Loading nuclei predictions from: {JSON_PREDICTIONS_PATH}")
with open(JSON_PREDICTIONS_PATH, 'r') as f:
    predictions_data = json.load(f)
print("Predictions loaded successfully.")

Loading nuclei predictions from: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/inference-result/instance_segmentation.json
Predictions loaded successfully.


In [25]:
total_images = len(predictions_data)
print(f"\nTotal Number of images processed : {total_images}")


Total Number of images processed : 10


In [26]:
for image_filename, nuclei_data in predictions_data.items():
    original_image_path = os.path.join(ORIGINAL_IMAGES_DIR, image_filename)
    output_image_path = os.path.join(OUTPUT_OVERLAY_DIR, f"overlay-legend_{image_filename}") 

    # Load the original image
    image = cv2.imread(original_image_path)

    if image is None:
        print(f"Warning: Could not load image {original_image_path}. Skipping.")
        continue

    # Create a copy of the original image to draw on to ensure the original image is not modified in memory
    final_image_with_overlay = image.copy() 

    print(f"  Processing {image_filename} with {len(nuclei_data)} nuclei...")

    # Iterate through each nucleus in the current image
    for nucleus_id, nucleus_info in nuclei_data.items():
        contour = nucleus_info.get('contour')
        nuclei_type = nucleus_info.get('type', 'UNK') # Default to 'UNK' if type is missing

        if contour is None or not contour:
            print(f"    Skipping nucleus {nucleus_id} in {image_filename}: No contour found.")
            continue

        contour_np = np.array(contour, dtype=np.int32)
        if contour_np.ndim == 2:
            contour_np = contour_np.reshape((-1, 1, 2))

        color = NUCLEI_TYPE_COLORS.get(nuclei_type, (128, 128, 128))

        cv2.drawContours(final_image_with_overlay, [contour_np], -1, color, 2) # thickness of 2 

    # --- Add Legend to the Image ---
    legend_start_x = image.shape[1] - 250 # Start 250 pixels from the right edge
    legend_start_y = 20 # Start 20 pixels from the top
    color_box_size = 20
    text_offset_x = color_box_size + 10
    line_height = 30
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 2
    text_color = (0, 0, 0) # Black text for the legend

    legend_bg_width = 230 # Approximate width needed for color box + text
    legend_bg_height = len(NUCLEI_TYPE_COLORS) * line_height + 20 # Height based on number of items
    legend_bg_rect_start = (legend_start_x - 10, legend_start_y - 10)
    legend_bg_rect_end = (legend_start_x - 10 + legend_bg_width, legend_start_y - 10 + legend_bg_height)

    # Create a translucent rectangle for the background
    sub_img = final_image_with_overlay[legend_bg_rect_start[1]:legend_bg_rect_end[1], legend_bg_rect_start[0]:legend_bg_rect_end[0]]
    white_rect = np.full(sub_img.shape, 255, dtype=np.uint8) # White background
    alpha_value = 0.5 # 50% transparency
    
    # Blend the white rectangle with the image region
    cv2.addWeighted(sub_img, 1 - alpha_value, white_rect, alpha_value, 0, sub_img)


    current_y = legend_start_y
    for nuclei_type, color_bgr in NUCLEI_TYPE_COLORS.items():
        # Draw color box
        cv2.rectangle(
            final_image_with_overlay,
            (legend_start_x, current_y),
            (legend_start_x + color_box_size, current_y + color_box_size),
            color_bgr,
            -1 
        )
        outline_color = (0, 0, 0) # Black outline
        cv2.putText(
            final_image_with_overlay,
            nuclei_type,
            (legend_start_x + text_offset_x, current_y + color_box_size - 5), # Adjust text position
            font,
            font_scale,
            outline_color,
            font_thickness,
            cv2.LINE_AA
        )
        current_y += line_height 

    # Save the overlaid image
    cv2.imwrite(output_image_path, final_image_with_overlay)
    print(f"Saved overlaid image: {output_image_path}")

  Processing image_01.png with 21 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_01.png
  Processing image_02.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_02.png
  Processing image_03.png with 1 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_03.png
  Processing image_04.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_04.png


  Processing image_05.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_05.png
  Processing image_06.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_06.png
  Processing image_07.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_07.png
  Processing image_08.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_08.png
  Processing image_09.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay/overlay-legend_image_09.png
  Processing image_10.png with 5 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/p

In [27]:
# Overlay without legend
OUTPUT_OVERLAY_DIR_NO_LEGEND = '/home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend'
os.makedirs(OUTPUT_OVERLAY_DIR_NO_LEGEND, exist_ok=True)

print(f"\nSaving overlay images without legend to: {OUTPUT_OVERLAY_DIR_NO_LEGEND}")


Saving overlay images without legend to: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend


In [28]:
for image_filename, nuclei_data in predictions_data.items():
    original_image_path = os.path.join(ORIGINAL_IMAGES_DIR, image_filename)
    output_image_path = os.path.join(OUTPUT_OVERLAY_DIR_NO_LEGEND, f"overlay-no-legend_{image_filename}") 

    # Load the original image
    image = cv2.imread(original_image_path)

    if image is None:
        print(f"Warning: Could not load image {original_image_path}. Skipping.")
        continue

    # Create a copy of the original image to draw on to ensure the original image is not modified in memory
    final_image_with_overlay = image.copy() 

    print(f"  Processing {image_filename} with {len(nuclei_data)} nuclei...")

    # Iterate through each nucleus in the current image
    for nucleus_id, nucleus_info in nuclei_data.items():
        contour = nucleus_info.get('contour')
        nuclei_type = nucleus_info.get('type', 'UNK') # Default to 'UNK' if type is missing

        if contour is None or not contour:
            print(f"    Skipping nucleus {nucleus_id} in {image_filename}: No contour found.")
            continue

        contour_np = np.array(contour, dtype=np.int32)
        if contour_np.ndim == 2:
            contour_np = contour_np.reshape((-1, 1, 2))

        color = NUCLEI_TYPE_COLORS.get(nuclei_type, (128, 128, 128))

        cv2.drawContours(final_image_with_overlay, [contour_np], -1, color, 2) # thickness of 2  

    # Save the overlaid image
    cv2.imwrite(output_image_path, final_image_with_overlay)
    print(f"Saved overlaid image: {output_image_path}")

  Processing image_01.png with 21 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_01.png


  Processing image_02.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_02.png
  Processing image_03.png with 1 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_03.png
  Processing image_04.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_04.png
  Processing image_05.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_05.png
  Processing image_06.png with 0 nuclei...
Saved overlaid image: /home/KutumLabGPU/Documents/santosh/TNBC-project/prediction-results/cellvit/overlay-no-legend/overlay-no-legend_image_06.png
  Processing image_07.png with 0 nuclei...
Saved o